### Сначала в ноутбуке проделал предобработку с padding, bag of words и проверил на Linear и DNN, эти ячейки закомментировал

### Steps поставил 10000 для ускорения сравнения моделей и подбора гиперпараметров

In [1]:
#Загружаем библиотеки
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import tqdm
import keras

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib

import pandas as pd
import re
from pymystem3 import Mystem
m = Mystem()

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

/anaconda3/envs/google/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/google/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


1.9.0


Using TensorFlow backend.


### Работа с данными (1 балл)

Загрузите датасет, с которым вы работали во время соревнования на kaggle. Преобразуйте его в формат, удобный для обучения модели. В качетсве фичей используйте эмбединги слов.

### Сначала напишем все необходимые функции и библиотеки для датасета, загрузки эмбедингов

In [2]:
#Загружаем эмбединги слов
import io
import gzip
import pathlib
import urllib.request

WORD2VEC_PATH = pathlib.Path('word2vec.bin')

if not WORD2VEC_PATH.exists():
    url = 'https://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz'
    with urllib.request.urlopen(url) as connection:
        compressed = connection.read()
            
    decompressed = gzip.GzipFile(fileobj=io.BytesIO(compressed), mode='rb').read()
    WORD2VEC_PATH.write_bytes(decompressed)

In [3]:
#Загружаем эмбединги слов
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_PATH, binary=True)

In [4]:
#Загружаем эмбединги слов
word2vec_mystem = Mystem(entire_input=False)


def tokenize_with_mystem_pos(text):
    result = []
    for item in word2vec_mystem.analyze(text):
        if item['analysis']:
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            token = f'{lemma}_{pos}'
        else:
            token = f'{item["text"]}_UNKN'
            
        result.append(token)

    return result

In [5]:
#Загружаем эмбединги слов
import numpy as np
from sklearn.base import TransformerMixin

class Word2VecVectorizer(TransformerMixin):
    def __init__(self, vectors):
        self.vectors = vectors
        self.zeros = np.zeros(self.vectors.vector_size)
        
    def _get_text_vector(self, text):
        token_vectors = []
        for token in tokenize_with_mystem_pos(text):
            try:
                token_vectors.append(self.vectors[token])
            except KeyError:
                pass
                
        if not token_vectors:
            return self.zeros
        return token_vectors
        #text_vector = np.sum(token_vectors, axis=0)
        #return text_vector / np.linalg.norm(text_vector)
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return np.array([self._get_text_vector(text) for text in X])

In [6]:
#Загружаем эмбединги слов
word2vec_vectorizer = Word2VecVectorizer(word2vec)

### Предобработка текста
- Специально с mystem.analyze, чтобы потом делать эмбединги

In [7]:
#Обработка текста
import razdel

def tokenize_with_razdel(text):
    return [token.text for token in razdel.tokenize(text)]

def my_tokenizer(text):
    
    tokens = ['<START>']
    #tokens= []
    
    text=text.lower()
    
    text = re.sub('[^а-яa-zё]+', ' ', text)
    text = re.sub('ё', 'е', text)
    text = re.sub('\s+', ' ', text)
    
    text = tokenize_with_mystem_pos(text)
    
    tokens = ' '.join(text)
    return tokens

In [8]:
#Загружаем данные
train = pd.read_csv('train.csv')

In [9]:
#Обработка текста
train['ready_text'] = train.text.apply(my_tokenizer)

In [10]:
train['ready_text'].head()

0              ж_PART позн_S с_S мужч_UNKN от_PR год_S
1    уже_ADV и_CONJ не_PART вспомнить_V где_ADVPRO ...
2      sep_UNKN decimal_UNKN может_ADV изза_S это_SPRO
3             нужный_A дева_S для_PR с_S о_S л_S смс_S
4    м_S искать_V д_S ж_CONJ с_PR п_S для_PR ласка_...
Name: ready_text, dtype: object

In [11]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train, test_size=0.2, stratify = train.label, random_state=42)

### Подготовка данных в модель

In [12]:
vocab_size = 5001
#vocab_size=5000

In [13]:
from keras.preprocessing.text import Tokenizer
#Создаем единый словарь (слово -> число) для преобразования
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>', filters='', lower=False)
#tokenizer.fit_on_texts(train.ready_text.tolist())
tokenizer.fit_on_texts(train_data.ready_text.tolist())

In [14]:
#Переводим тексты в последовательность индексов
#textSequences = tokenizer.texts_to_sequences(train.ready_text.tolist())
train_textSequences = tokenizer.texts_to_sequences(train_data.ready_text.tolist())
test_textSequences = tokenizer.texts_to_sequences(test_data.ready_text.tolist())

In [15]:
total_words = len(tokenizer.word_index)
print('В словаре {} слов'.format(total_words))

В словаре 51366 слов


In [16]:
#textSequences[0]

In [17]:
train_textSequences[0]

[12, 141, 2215, 4, 3264, 431]

In [18]:
test_textSequences[0]

[12, 896, 505, 2, 1, 5, 4569]

In [19]:
# Назначим словарь, который будет отображать слова из массива данных
word_index = tokenizer.word_index
word_index['<PAD>'] = 0
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
reverse_word_index[0] = '<PAD>'

In [20]:
# функция, которая сможет восстановить исходный текст
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [21]:
#decode_review(textSequences[7])
decode_review(train_textSequences[8])

'давать_V угадывать_V чтобы_CONJ кормить_V сетка_S'

In [22]:
decode_review(train_textSequences[8])

'давать_V угадывать_V чтобы_CONJ кормить_V сетка_S'

In [23]:
#Делаем паддинг

vocab_size = 5001
#vocab_size=5000
sentence_size = 200
embedding_size = 1000
model_dir = tempfile.mkdtemp()

# x_train = keras.preprocessing.sequence.pad_sequences(textSequences,
#                                                         value=0,
#                                                         truncating='post',
#                                                         padding='post',
#                                                         maxlen=sentence_size)

x_train = keras.preprocessing.sequence.pad_sequences(train_textSequences,
                                                        value=0,
                                                        truncating='post',
                                                        padding='post',
                                                        maxlen=sentence_size)

x_test = keras.preprocessing.sequence.pad_sequences(test_textSequences,
                                                        value=0,
                                                        truncating='post',
                                                        padding='post',
                                                        maxlen=sentence_size)

In [24]:
#y_train = list(train.label)
y_train = list(train_data.label)
y_test = list(test_data.label)

In [25]:
#x_train[5]
x_train[5]

array([  12,   28, 3771,   44,  563,   78,  116,   12,   28,    4, 2047,
        519,   12,   28,  113, 3771, 1297,   26,    3,  385,   28,   13,
          3,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [26]:
#textSequences[5]
train_textSequences[5]

[12,
 28,
 3771,
 44,
 563,
 78,
 116,
 12,
 28,
 4,
 2047,
 519,
 12,
 28,
 113,
 3771,
 1297,
 26,
 3,
 385,
 28,
 13,
 3,
 1]

In [27]:
#x_train_variable = textSequences
x_train_variable = train_textSequences
x_test_variable = test_textSequences

In [28]:
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [29]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=10000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['class_ids'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    #pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        #writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

### Проверка на LinearClassifier

In [30]:
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)
classifier = tf.estimator.LinearClassifier(feature_columns=[column], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpxlvvat05', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a56185860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
# train_and_evaluate(classifier)

In [32]:
# test_predictions = np.array([p['class_ids'][0] for p in classifier.predict(input_fn=eval_input_fn)])

In [33]:
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.metrics import classification_report
# print(np.mean(test_predictions == y_test))
# print(classification_report(y_true=y_test, y_pred=test_predictions))

### Проверка на DNNClassifier

In [34]:
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
classifier = tf.estimator.DNNClassifier(
    hidden_units=[100],
    feature_columns=[word_embedding_column], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpda6q1aha', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a56185dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [35]:
# word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
# classifier = tf.estimator.DNNClassifier(
#     hidden_units=[100],
#     feature_columns=[word_embedding_column], n_classes=3)
# train_and_evaluate(classifier)

In [36]:
# test_predictions = np.array([p['class_ids'][0] for p in classifier.predict(input_fn=eval_input_fn)])

In [37]:
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.metrics import classification_report
# print(np.mean(test_predictions == y_test))
# print(classification_report(y_true=y_test, y_pred=test_predictions))

### Проверка на CNN с лекции

In [38]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp9p2fubsw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a62763198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
# train_and_evaluate(cnn_classifier)

### Наконец, само задание, загрузка эмбедингов RusVectores
- Модель загружается выше, либо по файлу с первой лекции, либо по ссылке

In [40]:
def load_rusvec_embeddings(word2vec):
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size, embedding_size))
    num_loaded = 0
    for w, i in word_index.items():
        try:
            v = word2vec[w]
        except KeyError:
            v = None
        if v is not None and i < vocab_size:
            embedding_matrix[i] = v
            num_loaded += 1
        #if v is None:
        #    print(w)
    print('Successfully loaded pretrained embeddings for '
          f'{num_loaded}/{vocab_size} words.')
    embedding_matrix = embedding_matrix.astype(np.float32)
    return embedding_matrix

In [41]:
embedding_matrix = load_rusvec_embeddings(word2vec)

Successfully loaded pretrained embeddings for 4170/5001 words.


In [42]:
def my_initializer(shape=None, dtype=tf.float32, partition_info=None):
    assert dtype is tf.float32
    return embedding_matrix

In [43]:
params = {'embedding_initializer': my_initializer}

In [44]:
cnn_pretrained_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn_pretrained'),
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpeu036652/cnn_pretrained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a5b693f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
# train_and_evaluate(cnn_pretrained_classifier)

### Подготовили данные для обучения моделей
- В embedding_matrix лежат эмбединги RusVectores слов из словаря
- Что интересно, потеряли много важных слов, то есть много английских и специфические типа "коддинг"
- Следует брать другие эмбединги

### Переходим к моделям

### CNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- Dense

Параметры модели подберите сами.

### Две модели
- В первой после Conv1D идет reduce_max
- Во второй идет MaxPooling1D

In [46]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN

    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwhepyp1m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6beeb208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwhepyp1m/model.ckpt.
INFO:tensorflow:loss = 1.2241492, step = 1
INFO:tensorflow:global_step/sec: 1.88365
INFO:tensorflow:loss = 0.58988875, step = 101 (53.088 sec)
INFO:tensorflow:global_step/sec: 1.64355
INFO:tensorflow:loss = 0.553128, step = 201 (60.845 sec)
INFO:tensorflow:global_step/sec: 2.14603
INFO:tensorflow:loss = 0.5597145, step = 301 (46.597 sec)
INFO:tensorflow:global_step/sec: 2.19713
INFO:tensorflow:loss = 0.4691684, step = 401 (45.514 sec)
INFO:tensorflow:global_step/sec: 2.12319
INFO:tensorflow:loss = 0.33770755, step = 501 (47.100 sec)
INFO:tensorflow:global_step/sec: 2.17257
INFO:tensorflow:loss = 0.3768572, step = 601 (46.027 sec)
INFO:t

INFO:tensorflow:loss = 0.10248542, step = 7201 (50.534 sec)
INFO:tensorflow:global_step/sec: 1.82035
INFO:tensorflow:loss = 0.10543013, step = 7301 (54.935 sec)
INFO:tensorflow:global_step/sec: 1.90684
INFO:tensorflow:loss = 0.0854847, step = 7401 (52.443 sec)
INFO:tensorflow:global_step/sec: 1.87176
INFO:tensorflow:loss = 0.14921899, step = 7501 (53.425 sec)
INFO:tensorflow:global_step/sec: 1.99651
INFO:tensorflow:loss = 0.1166201, step = 7601 (50.087 sec)
INFO:tensorflow:global_step/sec: 2.09835
INFO:tensorflow:loss = 0.15304461, step = 7701 (47.657 sec)
INFO:tensorflow:Saving checkpoints for 7795 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwhepyp1m/model.ckpt.
INFO:tensorflow:global_step/sec: 2.0154
INFO:tensorflow:loss = 0.1089628, step = 7801 (49.618 sec)
INFO:tensorflow:global_step/sec: 1.94472
INFO:tensorflow:loss = 0.08840023, step = 7901 (51.421 sec)
INFO:tensorflow:global_step/sec: 2.16263
INFO:tensorflow:loss = 0.13802357, step = 8001 (46.240 sec)
INFO:tensorflo

In [48]:
test_predictions = np.array([p['class_ids'][0] for p in cnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwhepyp1m/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [49]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(np.mean(test_predictions == y_test))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.855917133457811
              precision    recall  f1-score   support

           0       0.70      0.63      0.67      4323
           1       0.97      0.97      0.97     10000
           2       0.80      0.83      0.81      8171

   micro avg       0.86      0.86      0.86     22494
   macro avg       0.82      0.81      0.82     22494
weighted avg       0.85      0.86      0.85     22494



In [50]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN

    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    # 1 200
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size=10, strides = 200, padding='same')
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwaxs5rxh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6cc1e128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwaxs5rxh/model.ckpt.
INFO:tensorflow:loss = 1.0626543, step = 1
INFO:tensorflow:global_step/sec: 2.2628
INFO:tensorflow:loss = 0.5872036, step = 101 (44.194 sec)
INFO:tensorflow:global_step/sec: 2.39777
INFO:tensorflow:loss = 0.4616033, step = 201 (41.706 sec)
INFO:tensorflow:global_step/sec: 2.40398
INFO:tensorflow:loss = 0.5774101, step = 301 (41.597 sec)
INFO:tensorflow:global_step/sec: 2.18881
INFO:tensorflow:loss = 0.45139152, step = 401 (45.687 sec)
INFO:tensorflow:global_step/sec: 2.32018
INFO:tensorflow:loss = 0.5186358, step = 501 (43.100 sec)
INFO:tensorflow:global_step/sec: 2.34694
INFO:tensorflow:loss = 0.39074394, step = 601 (42.608 sec)
INFO:t

INFO:tensorflow:global_step/sec: 2.32741
INFO:tensorflow:loss = 0.07938526, step = 7301 (42.967 sec)
INFO:tensorflow:global_step/sec: 2.26791
INFO:tensorflow:loss = 0.14647925, step = 7401 (44.094 sec)
INFO:tensorflow:global_step/sec: 2.27607
INFO:tensorflow:loss = 0.14796439, step = 7501 (43.935 sec)
INFO:tensorflow:global_step/sec: 2.28072
INFO:tensorflow:loss = 0.24681765, step = 7601 (43.846 sec)
INFO:tensorflow:global_step/sec: 2.27591
INFO:tensorflow:loss = 0.09472347, step = 7701 (43.939 sec)
INFO:tensorflow:global_step/sec: 2.32969
INFO:tensorflow:loss = 0.08452828, step = 7801 (42.924 sec)
INFO:tensorflow:global_step/sec: 2.33662
INFO:tensorflow:loss = 0.13928518, step = 7901 (42.797 sec)
INFO:tensorflow:Saving checkpoints for 7952 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwaxs5rxh/model.ckpt.
INFO:tensorflow:global_step/sec: 2.24801
INFO:tensorflow:loss = 0.15874813, step = 8001 (44.484 sec)
INFO:tensorflow:global_step/sec: 2.28056
INFO:tensorflow:loss = 0.0994

In [52]:
test_predictions = np.array([p['class_ids'][0] for p in cnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpwaxs5rxh/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [53]:
test_predictions = [test_predictions[i][0] for i in range(len(test_predictions))]

In [54]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8415132924335378
              precision    recall  f1-score   support

           0       0.64      0.66      0.65      4323
           1       0.97      0.97      0.97     10000
           2       0.80      0.78      0.79      8171

   micro avg       0.84      0.84      0.84     22494
   macro avg       0.80      0.80      0.80     22494
weighted avg       0.84      0.84      0.84     22494



### Вывод по CNN
- Результат хуже, чем на линейных моделях
- Нужно лучше подбирать параметры/поменять предобработку

### RNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- RNN
- Dense

Параметры модели подберите сами.

In [46]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)

def lstm_model_fn(features, labels, mode, params):    
    # [batch_size x sentence_size x embedding_size]
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])

    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(100)
    
    _, final_states = tf.nn.dynamic_rnn(
        rnn_cell, inputs, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states
    
    hidden = tf.layers.dense(inputs=outputs, units=200, activation=tf.nn.relu)

    logits = tf.layers.dense(inputs=hidden, units=3)

    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    optimizer = tf.train.AdamOptimizer()

    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)


params = {'embedding_initializer': my_initializer}
rnn_classifier = tf.estimator.Estimator(model_fn=lstm_model_fn,
                                         model_dir=os.path.join(model_dir, 'rnn'), params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp7owa50nj/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6bed2128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
train_and_evaluate(rnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp7owa50nj/rnn/model.ckpt.
INFO:tensorflow:loss = 1.0835376, step = 1
INFO:tensorflow:global_step/sec: 1.00679
INFO:tensorflow:loss = 0.5171749, step = 101 (99.327 sec)
INFO:tensorflow:global_step/sec: 1.23299
INFO:tensorflow:loss = 0.4076849, step = 201 (81.103 sec)
INFO:tensorflow:global_step/sec: 1.22162
INFO:tensorflow:loss = 0.5284697, step = 301 (81.859 sec)
INFO:tensorflow:global_step/sec: 1.12732
INFO:tensorflow:loss = 0.3464666, step = 401 (88.706 sec)
INFO:tensorflow:global_step/sec: 0.939479
INFO:tensorflow:loss = 0.36198482, step = 501 (106.451 sec)
INFO:tensorflow:global_step/sec: 1.13707
INFO:tensorflow:loss = 0.42667067, step = 601 (87.937 sec)

INFO:tensorflow:global_step/sec: 1.20447
INFO:tensorflow:loss = 0.14425737, step = 6601 (83.025 sec)
INFO:tensorflow:global_step/sec: 1.22419
INFO:tensorflow:loss = 0.1851988, step = 6701 (81.686 sec)
INFO:tensorflow:global_step/sec: 1.23551
INFO:tensorflow:loss = 0.1969355, step = 6801 (80.939 sec)
INFO:tensorflow:global_step/sec: 1.2129
INFO:tensorflow:loss = 0.35971528, step = 6901 (82.447 sec)
INFO:tensorflow:global_step/sec: 1.23413
INFO:tensorflow:loss = 0.17040098, step = 7001 (81.029 sec)
INFO:tensorflow:global_step/sec: 1.20894
INFO:tensorflow:loss = 0.23762746, step = 7101 (82.718 sec)
INFO:tensorflow:Saving checkpoints for 7125 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp7owa50nj/rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 1.19895
INFO:tensorflow:loss = 0.14277129, step = 7201 (83.406 sec)
INFO:tensorflow:global_step/sec: 1.2284
INFO:tensorflow:loss = 0.2630414, step = 7301 (81.407 sec)
INFO:tensorflow:global_step/sec: 1.20977
INFO:tensorflow:loss = 0.19256

In [48]:
test_predictions = np.array([p['class_ids'][0] for p in rnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp7owa50nj/rnn/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [49]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.857117453543167
              precision    recall  f1-score   support

           0       0.72      0.61      0.66      4323
           1       0.96      0.97      0.97     10000
           2       0.79      0.84      0.82      8171

   micro avg       0.86      0.86      0.86     22494
   macro avg       0.83      0.81      0.82     22494
weighted avg       0.85      0.86      0.85     22494



### LSTM model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- LSTM
- Dense

Параметры модели подберите сами.

In [55]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)

def lstm_model_fn(features, labels, mode):    
    # [batch_size x sentence_size x embedding_size]
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])

    # create an LSTM cell of size 100
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    
    # create the complete LSTM
    _, final_states = tf.nn.dynamic_rnn(
        lstm_cell, inputs, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states.h
    
    hidden = tf.layers.dense(inputs=outputs, units=200, activation=tf.nn.relu)

    logits = tf.layers.dense(inputs=hidden, units=3)

    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    optimizer = tf.train.AdamOptimizer()

    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)


params = {'embedding_initializer': my_initializer}
lstm_classifier = tf.estimator.Estimator(model_fn=lstm_model_fn,
                                         model_dir=os.path.join(model_dir, 'lstm'))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6c521cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [56]:
train_and_evaluate(lstm_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm/model.ckpt.
INFO:tensorflow:loss = 1.0990603, step = 1
INFO:tensorflow:global_step/sec: 0.876481
INFO:tensorflow:loss = 0.5057987, step = 101 (114.094 sec)
INFO:tensorflow:global_step/sec: 0.896092
INFO:tensorflow:loss = 0.27062434, step = 201 (111.595 sec)
INFO:tensorflow:global_step/sec: 0.893427
INFO:tensorflow:loss = 0.39828804, step = 301 (111.929 sec)
INFO:tensorflow:global_step/sec: 0.948081
INFO:tensorflow:loss = 0.33761868, step = 401 (105.476 sec)
INFO:tensorflow:global_step/sec: 0.896737
INFO:tensorflow:loss = 0.3

INFO:tensorflow:loss = 0.09841843, step = 6201 (114.881 sec)
INFO:tensorflow:Saving checkpoints for 6240 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm/model.ckpt.
INFO:tensorflow:global_step/sec: 0.867791
INFO:tensorflow:loss = 0.036425453, step = 6301 (115.235 sec)
INFO:tensorflow:global_step/sec: 0.859661
INFO:tensorflow:loss = 0.036516175, step = 6401 (116.325 sec)
INFO:tensorflow:global_step/sec: 0.866515
INFO:tensorflow:loss = 0.02058611, step = 6501 (115.405 sec)
INFO:tensorflow:global_step/sec: 0.884624
INFO:tensorflow:loss = 0.01994691, step = 6601 (113.042 sec)
INFO:tensorflow:global_step/sec: 0.851403
INFO:tensorflow:loss = 0.010634996, step = 6701 (117.453 sec)
INFO:tensorflow:Saving checkpoints for 6755 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm/model.ckpt.
INFO:tensorflow:global_step/sec: 0.817498
INFO:tensorflow:loss = 0.08498799, step = 6801 (122.325 sec)
INFO:tensorflow:global_step/sec: 0.815516
INFO:tensorflow:loss =

In [57]:
test_predictions = np.array([p['class_ids'][0] for p in lstm_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpij7kltm5/lstm/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [58]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8536943184849293
              precision    recall  f1-score   support

           0       0.67      0.66      0.67      4323
           1       0.97      0.97      0.97     10000
           2       0.80      0.81      0.81      8171

   micro avg       0.85      0.85      0.85     22494
   macro avg       0.82      0.81      0.82     22494
weighted avg       0.85      0.85      0.85     22494



Сравните все три реализованные модели по времени и по качеству классификации. Какая лучше? Как думаете, почему?

<b>Вывод:</b> CNN работает быстрее всего, потом дольше считается RNN, и дольше всех учится LSTM. LSTM учится примерно в два раза дольше, чем CNN. Ожидалось, что лучшее качество даст LSTM, потому что решает многие проблемы, например, затухание градиента.

- Лучший результат по Accuracy получили на RNN, лучший F1 получили на LSTM. То есть по выбранной метрике F1 LSTM показывает более хорошее качество.

- Вообще для более правильного сравнения еще следует менять число steps, потому что могло произойти переобучение

### Baseline model (2 балла)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Параметры для Conv1D, MaxPooling1D и RNN подберите сами.

In [50]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN

    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    # 1 200
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size=10, strides = 200, padding='same')
    
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(100)
    
    _, final_states = tf.nn.dynamic_rnn(
        rnn_cell, pool, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states
    
    hidden = tf.layers.dense(inputs=outputs, units=200, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
base_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp_6i3un3z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6c309550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
train_and_evaluate(base_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp_6i3un3z/model.ckpt.
INFO:tensorflow:loss = 1.2715847, step = 1
INFO:tensorflow:global_step/sec: 1.84227
INFO:tensorflow:loss = 0.68638474, step = 101 (54.282 sec)
INFO:tensorflow:global_step/sec: 2.02236
INFO:tensorflow:loss = 0.5124933, step = 201 (49.449 sec)
INFO:tensorflow:global_step/sec: 1.89951
INFO:tensorflow:loss = 0.3746278, step = 301 (52.643 sec)
INFO:tensorflow:global_step/sec: 2.37051
INFO:tensorflow:loss = 0.3940886, step = 401 (42.185 sec)
INFO:tensorflow:global_step/sec: 2.38317
INFO:tensorflow:loss = 0.41913098, step = 501 (41.961 sec)
INFO:tensorflow:global_step/sec: 2.33478
INFO:tensorflow:loss = 0.3436489, step = 601 (42.831 sec)
INFO:

INFO:tensorflow:global_step/sec: 2.35029
INFO:tensorflow:loss = 0.17347379, step = 7301 (42.548 sec)
INFO:tensorflow:global_step/sec: 2.45649
INFO:tensorflow:loss = 0.13547796, step = 7401 (40.708 sec)
INFO:tensorflow:global_step/sec: 2.46194
INFO:tensorflow:loss = 0.1076419, step = 7501 (40.619 sec)
INFO:tensorflow:global_step/sec: 2.45361
INFO:tensorflow:loss = 0.08217236, step = 7601 (40.756 sec)
INFO:tensorflow:global_step/sec: 2.44536
INFO:tensorflow:loss = 0.1323501, step = 7701 (40.894 sec)
INFO:tensorflow:global_step/sec: 2.45764
INFO:tensorflow:loss = 0.08596193, step = 7801 (40.691 sec)
INFO:tensorflow:global_step/sec: 2.45531
INFO:tensorflow:loss = 0.097725056, step = 7901 (40.727 sec)
INFO:tensorflow:global_step/sec: 2.46028
INFO:tensorflow:loss = 0.09014702, step = 8001 (40.646 sec)
INFO:tensorflow:global_step/sec: 2.47015
INFO:tensorflow:loss = 0.06456418, step = 8101 (40.484 sec)
INFO:tensorflow:global_step/sec: 2.49186
INFO:tensorflow:loss = 0.11249009, step = 8201 (40.

In [52]:
test_predictions = np.array([p['class_ids'][0] for p in base_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp_6i3un3z/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.847425980261403
              precision    recall  f1-score   support

           0       0.68      0.61      0.64      4323
           1       0.97      0.97      0.97     10000
           2       0.78      0.82      0.80      8171

   micro avg       0.85      0.85      0.85     22494
   macro avg       0.81      0.80      0.80     22494
weighted avg       0.85      0.85      0.85     22494



### Dropout (1 балл)

Добавьте dropout к baseline моделе. Пример модефицированной модели:
- Dropout
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Подберите параметры.

In [56]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params): 
    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.1, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    # 1 200
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size=10, strides = 200, padding='same')
    
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(100)
    
    _, final_states = tf.nn.dynamic_rnn(
        rnn_cell, pool, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states
    
    hidden = tf.layers.dense(inputs=outputs, units=200, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
base_drop_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpuamtubq6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a6c1f0358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [57]:
train_and_evaluate(base_drop_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpuamtubq6/model.ckpt.
INFO:tensorflow:loss = 1.0694867, step = 1
INFO:tensorflow:global_step/sec: 1.59995
INFO:tensorflow:loss = 0.55550474, step = 101 (62.503 sec)
INFO:tensorflow:global_step/sec: 1.65385
INFO:tensorflow:loss = 0.487218, step = 201 (60.465 sec)
INFO:tensorflow:global_step/sec: 1.77118
INFO:tensorflow:loss = 0.423622, step = 301 (56.459 sec)
INFO:tensorflow:global_step/sec: 1.67685
INFO:tensorflow:loss = 0.32779527, step = 401 (59.636 sec)
INFO:tensorflow:global_step/sec: 1.85932
INFO:tensorflow:loss = 0.40464732, step = 501 (53.783 sec)
INFO:tensorflow:global_step/sec: 1.85347
INFO:tensorflow:loss = 0.39251357, step = 601 (53.953 sec)
INFO:

INFO:tensorflow:loss = 0.17233467, step = 7101 (58.731 sec)
INFO:tensorflow:global_step/sec: 1.75215
INFO:tensorflow:loss = 0.13001274, step = 7201 (57.073 sec)
INFO:tensorflow:global_step/sec: 1.75612
INFO:tensorflow:loss = 0.103261545, step = 7301 (56.944 sec)
INFO:tensorflow:global_step/sec: 1.68329
INFO:tensorflow:loss = 0.19734322, step = 7401 (59.409 sec)
INFO:tensorflow:global_step/sec: 1.64921
INFO:tensorflow:loss = 0.13563403, step = 7501 (60.634 sec)
INFO:tensorflow:Saving checkpoints for 7514 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpuamtubq6/model.ckpt.
INFO:tensorflow:global_step/sec: 1.68033
INFO:tensorflow:loss = 0.17108637, step = 7601 (59.512 sec)
INFO:tensorflow:global_step/sec: 1.74338
INFO:tensorflow:loss = 0.116622105, step = 7701 (57.360 sec)
INFO:tensorflow:global_step/sec: 1.74899
INFO:tensorflow:loss = 0.1037425, step = 7801 (57.194 sec)
INFO:tensorflow:global_step/sec: 1.66417
INFO:tensorflow:loss = 0.18452363, step = 7901 (60.071 sec)
INFO:tens

In [58]:
test_predictions = np.array([p['class_ids'][0] for p in base_drop_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpuamtubq6/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [59]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8490708633413354
              precision    recall  f1-score   support

           0       0.70      0.57      0.63      4323
           1       0.97      0.97      0.97     10000
           2       0.77      0.85      0.81      8171

   micro avg       0.85      0.85      0.85     22494
   macro avg       0.81      0.80      0.80     22494
weighted avg       0.85      0.85      0.85     22494



Улучшило ли результат классификации использование Dropout? Как думаете, почему?

<b>Вывод:</b> Улучшило. Dropout предназначен для решения проблемы переобучения в нейронных сетях. В данной архитектуре мы по сути выключаем некоторые эмбединги слов, делая модель более гибкой.

### Bidirectional (1 балл)

Вместо RNN в первой модели используйте biRNN. Пример модефецированной модели:
- Conv1D, activation='relu'
- MaxPooling1D
- biRNN
- Dense

Подберите параметры.

In [46]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN

    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size=10, strides = 200, padding='same')
    
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(50)
    
    outputs, final_states = tf.nn.bidirectional_dynamic_rnn(
        cell_fw = rnn_cell, cell_bw = rnn_cell, inputs = pool, dtype=tf.float32)
    
    output_state_fw, output_state_bw = final_states
    out_fw, out_bw = outputs

    out = tf.concat(outputs, 2)
    
    hidden = tf.layers.dense(inputs=out, units=200, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
birnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpncl2b09c', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a61550d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
train_and_evaluate(birnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpncl2b09c/model.ckpt.
INFO:tensorflow:loss = 1.0784558, step = 1
INFO:tensorflow:global_step/sec: 2.12354
INFO:tensorflow:loss = 0.59304655, step = 101 (47.091 sec)
INFO:tensorflow:global_step/sec: 2.25588
INFO:tensorflow:loss = 0.49763379, step = 201 (44.329 sec)
INFO:tensorflow:global_step/sec: 2.24858
INFO:tensorflow:loss = 0.5605636, step = 301 (44.472 sec)
INFO:tensorflow:global_step/sec: 2.28305
INFO:tensorflow:loss = 0.49010903, step = 401 (43.801 sec)
INFO:tensorflow:global_step/sec: 2.28564
INFO:tensorflow:loss = 0.5562684, step = 501 (43.752 sec)
INFO:tensorflow:global_step/sec: 2.29702
INFO:tensorflow:loss = 0.4237946, step = 601 (43.534 sec)
INFO

INFO:tensorflow:global_step/sec: 2.24438
INFO:tensorflow:loss = 0.13422573, step = 7301 (44.555 sec)
INFO:tensorflow:global_step/sec: 2.10905
INFO:tensorflow:loss = 0.090715714, step = 7401 (47.415 sec)
INFO:tensorflow:global_step/sec: 2.22812
INFO:tensorflow:loss = 0.16307156, step = 7501 (44.880 sec)
INFO:tensorflow:global_step/sec: 2.15812
INFO:tensorflow:loss = 0.096256234, step = 7601 (46.337 sec)
INFO:tensorflow:global_step/sec: 2.16919
INFO:tensorflow:loss = 0.105710946, step = 7701 (46.100 sec)
INFO:tensorflow:global_step/sec: 2.15151
INFO:tensorflow:loss = 0.20725945, step = 7801 (46.478 sec)
INFO:tensorflow:global_step/sec: 2.20353
INFO:tensorflow:loss = 0.12573558, step = 7901 (45.382 sec)
INFO:tensorflow:Saving checkpoints for 7998 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpncl2b09c/model.ckpt.
INFO:tensorflow:global_step/sec: 2.14625
INFO:tensorflow:loss = 0.16638294, step = 8001 (46.593 sec)
INFO:tensorflow:global_step/sec: 2.26105
INFO:tensorflow:loss = 0.0

In [48]:
test_predictions = np.array([p['class_ids'][0] for p in birnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpncl2b09c/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [49]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8496932515337423
              precision    recall  f1-score   support

           0       0.68      0.62      0.65      4323
           1       0.97      0.97      0.97     10000
           2       0.79      0.82      0.80      8171

   micro avg       0.85      0.85      0.85     22494
   macro avg       0.81      0.81      0.81     22494
weighted avg       0.85      0.85      0.85     22494



### Что-то не так со скоростью, так как последние шаги проходят сильно дольше
- Немного изменим модель и исправим это, ведь так не должно быть

In [50]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN

    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size=10, strides = 200, padding='same')
    
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(50)
    
    outputs, final_states = tf.nn.bidirectional_dynamic_rnn(
        cell_fw = rnn_cell, cell_bw = rnn_cell, inputs = pool, dtype=tf.float32)
    
    output_state_fw, output_state_bw = final_states
    out_fw, out_bw = outputs

    out = tf.concat(final_states, -1)
    
    hidden = tf.layers.dense(inputs=out, units=200, activation=tf.nn.relu)
    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
birnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpgpgjqu1o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a68d51a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
train_and_evaluate(birnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpgpgjqu1o/model.ckpt.
INFO:tensorflow:loss = 1.0908378, step = 1
INFO:tensorflow:global_step/sec: 1.81457
INFO:tensorflow:loss = 0.586916, step = 101 (55.111 sec)
INFO:tensorflow:global_step/sec: 1.76767
INFO:tensorflow:loss = 0.41730317, step = 201 (56.572 sec)
INFO:tensorflow:global_step/sec: 1.98968
INFO:tensorflow:loss = 0.43886513, step = 301 (50.259 sec)
INFO:tensorflow:global_step/sec: 2.18085
INFO:tensorflow:loss = 0.37238103, step = 401 (45.854 sec)
INFO:tensorflow:global_step/sec: 2.18976
INFO:tensorflow:loss = 0.4414874, step = 501 (45.667 sec)
INFO:tensorflow:global_step/sec: 2.21897
INFO:tensorflow:loss = 0.36375982, step = 601 (45.066 sec)
INFO

INFO:tensorflow:loss = 0.16744046, step = 7101 (53.358 sec)
INFO:tensorflow:global_step/sec: 1.89503
INFO:tensorflow:loss = 0.12736607, step = 7201 (52.771 sec)
INFO:tensorflow:global_step/sec: 1.68198
INFO:tensorflow:loss = 0.076178916, step = 7301 (59.453 sec)
INFO:tensorflow:global_step/sec: 1.8254
INFO:tensorflow:loss = 0.08865547, step = 7401 (54.782 sec)
INFO:tensorflow:global_step/sec: 1.88633
INFO:tensorflow:loss = 0.1655165, step = 7501 (53.013 sec)
INFO:tensorflow:global_step/sec: 1.92297
INFO:tensorflow:loss = 0.12907773, step = 7601 (52.003 sec)
INFO:tensorflow:global_step/sec: 1.83309
INFO:tensorflow:loss = 0.15415356, step = 7701 (54.555 sec)
INFO:tensorflow:global_step/sec: 1.94742
INFO:tensorflow:loss = 0.15606916, step = 7801 (51.348 sec)
INFO:tensorflow:Saving checkpoints for 7838 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpgpgjqu1o/model.ckpt.
INFO:tensorflow:global_step/sec: 1.55902
INFO:tensorflow:loss = 0.1590415, step = 7901 (64.143 sec)
INFO:tensorf

In [52]:
test_predictions = np.array([p['class_ids'][0] for p in birnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmpgpgjqu1o/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8476038054592335
              precision    recall  f1-score   support

           0       0.68      0.60      0.64      4323
           1       0.97      0.97      0.97     10000
           2       0.78      0.83      0.80      8171

   micro avg       0.85      0.85      0.85     22494
   macro avg       0.81      0.80      0.80     22494
weighted avg       0.85      0.85      0.85     22494



Дало ли буст использовании biRNN вместо RNN? Сильный? Как думаете, почему?

<b>Вывод:</b> Дало. Небольшой буст. Потому что так мы учитываем еще и последовательность слов в обратном порядке и последний слой дает информацию и о прямом проходе, и о обратном.

### Custom model (2 балла)

Подберите архитектуру сети, используя сверточные слои и слои макспулинга и превзойдите качество полносвязной сети.

Подберите архитектуру сети, используя следующие типы слоев:
- Conv1D
- MaxPooling1D
- RNN
- LSTM
- GRU
- Dropout
- Dense

Можно использовать любые из описанных слоев в любом количестве и в любом порядке. Не обязательно использовать все описанные слои. 

Настройте параметры. Превзойдите лучшее качество, полученное раннее. 

### Из построенных моделей лучший результат дает
- Модель RNN
- На ней имеем результат
- Accuracy: 0.857117453543167
- F1-score: 0.86

### Подберем архитектуру сети, чтобы улучшить результат
- Затем проверим качество на отложенной выборке,
- то есть на данных первого соревнования Kaggle

### Используется следующая архитектура
- Dropout
- GRU
- Dense
- Dropout

In [46]:
head = tf.contrib.estimator.multi_class_head(n_classes=3)
def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)
    
    rnn_cell = tf.nn.rnn_cell.GRUCell(100)
    
    _, final_states = tf.nn.dynamic_rnn(
        rnn_cell, dropout_emb, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states
    
    hidden = tf.layers.dense(inputs=outputs, units=200, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=3)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': my_initializer}
birnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a62784860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
train_and_evaluate(birnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt.
INFO:tensorflow:loss = 1.1135817, step = 1
INFO:tensorflow:global_step/sec: 0.80114
INFO:tensorflow:loss = 0.43878323, step = 101 (124.822 sec)
INFO:tensorflow:global_step/sec: 0.800399
INFO:tensorflow:loss = 0.4307265, step = 201 (124.938 sec)
INFO:tensorflow:global_step/sec: 0.830925
INFO:tensorflow:loss = 0.45533887, step = 301 (120.348 sec)
INFO:tensorflow:global_step/sec: 0.813355
INFO:tensorflow:loss = 0.24447277, step = 401 (122.947 sec)
INFO:tensorflow:Saving checkpoints for 485 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt.
INFO:tensorflow:global_step/sec: 0.794396
INFO:tensorflow:loss = 0.2596906

INFO:tensorflow:global_step/sec: 0.814814
INFO:tensorflow:loss = 0.046367828, step = 6301 (122.727 sec)
INFO:tensorflow:Saving checkpoints for 6380 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt.
INFO:tensorflow:global_step/sec: 0.810731
INFO:tensorflow:loss = 0.0652002, step = 6401 (123.353 sec)
INFO:tensorflow:global_step/sec: 0.818125
INFO:tensorflow:loss = 0.053018644, step = 6501 (122.223 sec)
INFO:tensorflow:global_step/sec: 0.855426
INFO:tensorflow:loss = 0.092910975, step = 6601 (116.901 sec)
INFO:tensorflow:global_step/sec: 0.806773
INFO:tensorflow:loss = 0.043586172, step = 6701 (123.951 sec)
INFO:tensorflow:global_step/sec: 0.808187
INFO:tensorflow:loss = 0.060973242, step = 6801 (123.734 sec)
INFO:tensorflow:Saving checkpoints for 6867 into /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt.
INFO:tensorflow:global_step/sec: 0.769123
INFO:tensorflow:loss = 0.08341308, step = 6901 (130.018 sec)
INFO:tensorflow:global_step/sec

In [48]:
test_predictions = np.array([p['class_ids'][0] for p in birnn_classifier.predict(input_fn=eval_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [49]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
print(accuracy_score(y_true = y_test, y_pred = test_predictions))
print(classification_report(y_true=y_test, y_pred=test_predictions))

0.8605405886014048
              precision    recall  f1-score   support

           0       0.69      0.67      0.68      4323
           1       0.97      0.97      0.97     10000
           2       0.81      0.83      0.82      8171

   micro avg       0.86      0.86      0.86     22494
   macro avg       0.83      0.82      0.82     22494
weighted avg       0.86      0.86      0.86     22494



### Получили лучший accuracy и выиграли в f1-score
- Это можно увидеть, посмотрев на показатели отдельно каждого класса

### Проверим качество на данных test соревнования Kaggle
- Сделаем predict на обученной модели (конечно, следовало обучить модель заново на всех данных train, а не на split train выборке)

In [70]:
# def print_predictions(sentences, classifier):
#     indexes = [text_to_index(sentence) for sentence in sentences]
#     x = sequence.pad_sequences(indexes,
#                                maxlen=sentence_size, 
#                                padding='post', 
#                                value=-1)
#     length = np.array([min(len(x), sentence_size) for x in indexes])
#     predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x, "len": length}, shuffle=False)
#     predictions = [p['logistic'][0] for p in classifier.predict(input_fn=predict_input_fn)]
#     print(predictions)

In [50]:
test_kaggle = pd.read_csv('test.csv')

In [51]:
#Обработка текста
test_kaggle['ready_text'] = test_kaggle.text.apply(my_tokenizer)

In [52]:
test_kaggle_textSequences = tokenizer.texts_to_sequences(test_kaggle.ready_text.tolist())

In [53]:
x_kaggle_test = keras.preprocessing.sequence.pad_sequences(test_kaggle_textSequences,
                                                        value=0,
                                                        truncating='post',
                                                        padding='post',
                                                        maxlen=sentence_size)

In [55]:
x_len_kaggle_test = np.array([min(len(x), sentence_size) for x in test_kaggle_textSequences])

In [67]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_kaggle_test, "len": x_len_kaggle_test}, shuffle=False)

In [68]:
kaggle_predictions = np.array([p['class_ids'][0] for p in birnn_classifier.predict(input_fn=predict_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mf/mbbt4y5524n7dzzccbd012h00000gn/T/tmp77h93eh9/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [73]:
test_kaggle['label'] = kaggle_predictions

In [74]:
submit = test_kaggle[['id', 'label']]

In [75]:
submit.to_csv('submit_gru.csv',index=False)

### Результат модели на Kaggle:
- 0.85782
- Довольно средне

### Что следовало сделать лучше и что могло улучшить результат
- Подобрать лучше эмбединги (например, взять другой с RusVectores)
- Лучше перебирать параметры
- Лучше сделать предобработку (тут использовал упрощенную предобработку, а не как в соревновании)
- Поставить padding значительно меньше (+- 50)
- Варьировать число шагов
- Добавлять/не добавлять start, oov (в каждом тексте сейчас есть start и oov, следует потестировать без них)
- Maru
#### Следует учесть это в соревновании DSSM